<a href="https://colab.research.google.com/github/harshil0217/NFL_Rookie_Comps/blob/main/ProfileEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import auth
from google.cloud import storage
import io
!pip install sentence-transformers
import spacy
!python -m spacy download en_core_web_sm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
auth.authenticate_user()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
client = storage.Client(project='cmse-381')
bucket = client.get_bucket('stats_draftprospects')
blob = bucket.blob('draft_profiles.csv')
content = blob.download_as_string()
profiles = pd.read_csv(io.BytesIO(content))

In [3]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,Brown has played second fiddle to Carnell Will...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,Williams started two games and played in nine ...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,Jefferson is an efficient back who follows and...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,Stuard is an undersized off-the-ball linebacke...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [4]:
#use BERT-NER to remove Person, Organization, Place, Date, and Time

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=device, batch_size=32, aggregation_strategy="simple")

def remove_entities(text):
    entities = nlp(text)
    for entity in entities:
      text = text.replace(entity['word'], '')
    return text

profiles['draft_profile'] = profiles['draft_profile'].apply(remove_entities)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing

In [5]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of head coach . attempted only five ...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,has played second fiddle to nell throughout ...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"' father, , played at and in the with the a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,was drafted by the in the 12th round out of ...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,started two games and played in nine in 2001 ...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4021,Jermar Jefferson,is an efficient back who follows and reads bl...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4022,Dax Milne,is a savvy route runner with average burst an...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Grant Stuard,uard is an undersized off-the-ball linebacker ...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Robert Jones,started 13 games at guard at the junior colle...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [6]:
# add sentiment co

In [7]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
embeddings = model.encode(profiles['draft_profile'].tolist())

In [9]:
embeddings

array([[ 0.00627863,  0.01658658, -0.08717795, ...,  0.03496552,
         0.01262207,  0.04706078],
       [-0.04669457, -0.04584552, -0.05577981, ..., -0.05562514,
         0.00264242,  0.04683511],
       [-0.03706425, -0.06409957, -0.03982502, ..., -0.00861998,
        -0.03117021,  0.05337487],
       ...,
       [ 0.00376847, -0.01687002, -0.03105178, ..., -0.02508315,
         0.02738197,  0.01849411],
       [-0.0118092 ,  0.05310997, -0.11014616, ..., -0.08132783,
        -0.03335277,  0.05237626],
       [-0.0297422 ,  0.04647798, -0.07648223, ..., -0.01655806,
        -0.00338332,  0.05657864]], dtype=float32)

In [10]:
similarity_matrix = cosine_similarity(embeddings)








In [11]:
similarity_df = pd.DataFrame(similarity_matrix, index=profiles['player_name'], columns=profiles['player_name'])

In [12]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,1.000000,0.486271,0.599639,0.624179,0.666452,0.635041,0.591860,0.575120,0.749781,0.732079,...,0.498793,0.446576,0.465359,0.483872,0.546248,0.605571,0.416939,0.351328,0.566784,0.457446
Ronnie Brown,0.486271,1.000000,0.601567,0.590608,0.610077,0.629863,0.582154,0.512900,0.691896,0.640026,...,0.471143,0.448868,0.458000,0.479595,0.490184,0.527700,0.453241,0.396308,0.505863,0.443285
Braylon Edwards,0.599639,0.601567,1.000000,0.609455,0.581892,0.626384,0.657132,0.501731,0.675475,0.629566,...,0.481412,0.459245,0.459547,0.514428,0.528010,0.576819,0.442471,0.324628,0.522491,0.432205
Cedric Benson,0.624179,0.590608,0.609455,1.000000,0.737377,0.668886,0.661113,0.568278,0.737842,0.688823,...,0.511234,0.614603,0.438515,0.560770,0.518957,0.603133,0.641885,0.441516,0.632538,0.641900
Carnell Williams,0.666452,0.610077,0.581892,0.737377,1.000000,0.637527,0.696843,0.584900,0.751220,0.753398,...,0.560373,0.556257,0.557769,0.559191,0.576297,0.637734,0.577659,0.463875,0.699827,0.638897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.605571,0.527700,0.576819,0.603133,0.637734,0.711190,0.525414,0.595233,0.661981,0.596492,...,0.650762,0.618511,0.514722,0.677498,0.740429,1.000000,0.683707,0.542741,0.669568,0.671905
Dax Milne,0.416939,0.453241,0.442471,0.641885,0.577659,0.515299,0.481421,0.455986,0.543337,0.550799,...,0.440945,0.707487,0.332505,0.659687,0.647144,0.683707,1.000000,0.540840,0.679666,0.841564
Grant Stuard,0.351328,0.396308,0.324628,0.441516,0.463875,0.465165,0.323200,0.406704,0.424986,0.424448,...,0.344506,0.521120,0.310592,0.464280,0.473990,0.542741,0.540840,1.000000,0.498922,0.552782


In [13]:
similarity_df.sort_values(by = 'Joe Burrow', ascending = False).head(10)


player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Joe Burrow,0.647075,0.579435,0.586340,0.606334,0.569060,0.689389,0.547496,0.598673,0.683158,0.664241,...,0.568271,0.638481,0.537479,0.672807,0.646605,0.731745,0.623343,0.487475,0.618668,0.617058
Nate Stanley,0.707439,0.645398,0.612162,0.683070,0.647217,0.723756,0.601335,0.596132,0.799788,0.759363,...,0.565317,0.628153,0.578509,0.665361,0.652589,0.703402,0.584719,0.491843,0.652930,0.577231
Ian Book,0.646829,0.524818,0.557835,0.617232,0.604240,0.649182,0.509711,0.572057,0.662570,0.638308,...,0.568753,0.670543,0.527989,0.633693,0.702570,0.774465,0.675836,0.550637,0.670577,0.672414
Trevor Lawrence,0.630071,0.526100,0.570410,0.693643,0.624470,0.672450,0.563878,0.544786,0.657754,0.682618,...,0.530596,0.688233,0.533944,0.677173,0.634396,0.739979,0.713510,0.551294,0.660619,0.735148
Dwayne Haskins,0.661748,0.648731,0.651097,0.686101,0.660198,0.700243,0.631670,0.593289,0.787714,0.740395,...,0.562385,0.642891,0.534451,0.675050,0.727460,0.726914,0.677757,0.488000,0.684545,0.643677
Cole McDonald,0.693110,0.611862,0.615921,0.663871,0.649716,0.639116,0.621504,0.540332,0.750202,0.731141,...,0.509792,0.569805,0.520099,0.632016,0.512172,0.633498,0.559373,0.447728,0.640533,0.580874
Marlon Davidson,0.646830,0.623711,0.599888,0.706115,0.705072,0.754496,0.643666,0.648435,0.753379,0.708636,...,0.617758,0.732181,0.575549,0.694942,0.674143,0.765391,0.729426,0.546543,0.767836,0.716339
Drew Lock,0.710628,0.549476,0.603692,0.647034,0.653146,0.695854,0.578758,0.598058,0.721966,0.703729,...,0.549424,0.600236,0.536537,0.682503,0.673912,0.736861,0.598057,0.449403,0.682065,0.639665
D'Andre Walker,0.617954,0.587666,0.568560,0.697824,0.655966,0.680377,0.582718,0.582620,0.699904,0.716597,...,0.601377,0.695388,0.500304,0.712253,0.652034,0.721606,0.673401,0.531762,0.708727,0.716015


In [14]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,1.000000,0.486271,0.599639,0.624179,0.666452,0.635041,0.591860,0.575120,0.749781,0.732079,...,0.498793,0.446576,0.465359,0.483872,0.546248,0.605571,0.416939,0.351328,0.566784,0.457446
Ronnie Brown,0.486271,1.000000,0.601567,0.590608,0.610077,0.629863,0.582154,0.512900,0.691896,0.640026,...,0.471143,0.448868,0.458000,0.479595,0.490184,0.527700,0.453241,0.396308,0.505863,0.443285
Braylon Edwards,0.599639,0.601567,1.000000,0.609455,0.581892,0.626384,0.657132,0.501731,0.675475,0.629566,...,0.481412,0.459245,0.459547,0.514428,0.528010,0.576819,0.442471,0.324628,0.522491,0.432205
Cedric Benson,0.624179,0.590608,0.609455,1.000000,0.737377,0.668886,0.661113,0.568278,0.737842,0.688823,...,0.511234,0.614603,0.438515,0.560770,0.518957,0.603133,0.641885,0.441516,0.632538,0.641900
Carnell Williams,0.666452,0.610077,0.581892,0.737377,1.000000,0.637527,0.696843,0.584900,0.751220,0.753398,...,0.560373,0.556257,0.557769,0.559191,0.576297,0.637734,0.577659,0.463875,0.699827,0.638897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.605571,0.527700,0.576819,0.603133,0.637734,0.711190,0.525414,0.595233,0.661981,0.596492,...,0.650762,0.618511,0.514722,0.677498,0.740429,1.000000,0.683707,0.542741,0.669568,0.671905
Dax Milne,0.416939,0.453241,0.442471,0.641885,0.577659,0.515299,0.481421,0.455986,0.543337,0.550799,...,0.440945,0.707487,0.332505,0.659687,0.647144,0.683707,1.000000,0.540840,0.679666,0.841564
Grant Stuard,0.351328,0.396308,0.324628,0.441516,0.463875,0.465165,0.323200,0.406704,0.424986,0.424448,...,0.344506,0.521120,0.310592,0.464280,0.473990,0.542741,0.540840,1.000000,0.498922,0.552782


In [15]:
joe_shiesty = similarity_df.loc['Joe Burrow']

In [18]:
joe_shiesty.sort_values(ascending = False).head(20)

,Joe Burrow
player_name,
Joe Burrow,1.000000
Nate Stanley,0.851501
Ian Book,0.842767
Trevor Lawrence,0.835453
Dwayne Haskins,0.827700
Cole McDonald,0.826031
Marlon Davidson,0.824697
Drew Lock,0.821436
D'Andre Walker,0.821323


In [17]:
# generate the three most similar players to Joe Burrow, making sure they match his position and were drafted in the same round

